<a href="https://colab.research.google.com/github/Newborn1937/zdetect/blob/main/zdetector_rand_avatars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Load avatars

In [ ]:
import gdown

#data_name = 'random_1e6'
#data_url = 'GDRIVE_LINK'

data_name = 'random_around_24Feb'
data_url = 'GDRIVE_LINK'

# Download an archive with avatars
arch_name = data_name + '.zip'
gdown.download(data_url, arch_name)

# Unzip the archive
!unzip -q /content/$arch_name

Downloading...
From: GDRIVE_LINKTo: /content/random_around_24Feb.zip
100%|██████████| 1.24G/1.24G [00:08<00:00, 142MB/s]


replace random_around_24Feb/679000188.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace random_around_24Feb/679000613.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace random_around_24Feb/679003657.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace random_around_24Feb/679003657.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A



### 2a. Classify avatars

In [ ]:
import gdown

# Load utility for high-level interaction with VK (unused here)
#url = 'REPO/vk_utils.py'
#gdown.download(url)

# Load utility for image classification
url = 'REPO/z_classifier.py'
gdown.download(url)

# Load the model trained to detect z-pictures
url = 'GDRIVE_LINK'
gdown.download(url, 'zconfig.json')
url = 'GDRIVE_LINK'
gdown.download(url, 'zmodel.pt')

Downloading...
From: REPO/z_classifier.py
To: /content/z_classifier.py
1.31kB [00:00, 527kB/s]                  
Downloading...
From: GDRIVE_LINKTo: /content/zconfig.json
100%|██████████| 78.0/78.0 [00:00<00:00, 79.6kB/s]
Downloading...
From: GDRIVE_LINKTo: /content/zmodel.pt
100%|██████████| 44.8M/44.8M [00:01<00:00, 30.9MB/s]


'zmodel.pt'

In [ ]:
from z_classifier import ZClassifier

# Create classifier
clf = ZClassifier('/content/')


In [ ]:
import os

from PIL import Image
from tqdm import tqdm

IMG_PATH = os.path.join('/content', data_name)
filenames = os.listdir(IMG_PATH)

res = []
err_list = []

# Classify images
for n, fname in enumerate(tqdm(filenames)):
  try:
    fpath = os.path.join(IMG_PATH, fname)
    img = Image.open(fpath)
    pvec = clf.predict(img)
    res.append([fname] + pvec.tolist())
    #if n > 2000: break
  except Exception as e:
    #print(f'{fname}: {e}')
    err_list.append([fname, e])


100%|██████████| 64721/64721 [20:33<00:00, 52.46it/s]


In [ ]:
from google.colab import files
import pandas as pd

columns = ['Filename'] + clf._classes
res_pd = pd.DataFrame(res, columns=columns)

#res_pd.to_pickle('clf_result.pkl')
fname_res = data_name + '_clf_res.csv'
res_pd.to_csv(fname_res)

#files.download('clf_result.pkl')
files.download(fname_res)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 2b. Upload previously obtained classification results

In [ ]:
from google.colab import files
import pandas as pd

fname_res = data_name + '_clf_res.csv'

# Upload classification result from local disk (csv)
print(f'Provide location of {fname_res}')
files.upload()

# Load table from csv
res_pd = pd.read_csv(os.path.join('/content', fname_res))

Saving clf_result.csv to clf_result.csv


### 3. Analyze classification result

In [ ]:
import numpy as np

th = 0.65
mask_0 = np.array(res_pd['z'] + res_pd['tricolor']) > th
mask_z = np.array(res_pd['z']) > np.array(res_pd['tricolor'])
mask_tri = np.array(res_pd['z']) <= np.array(res_pd['tricolor'])
mask_z = mask_z & mask_0
mask_tri = mask_tri & mask_0
idx = np.argwhere(mask_0)
len(idx)

924

In [ ]:
import os
import shutil

data_name_z = data_name + '_z'
data_name_tri = data_name + '_tricolor'

dirpath_in = os.path.join('/content', data_name)
dirpath_out_z = os.path.join('/content', data_name_z)
dirpath_out_tri = os.path.join('/content', data_name_tri)

res_pd_z = res_pd[mask_z]
res_pd_tri = res_pd[mask_tri]

res_pd_all = (res_pd_z, res_pd_tri)
dirpath_out_all = (dirpath_out_z, dirpath_out_tri)

for res_pd_, dirpath_out_ in zip(res_pd_all, dirpath_out_all):

  # Create folder for a class (z / tricolor)
  if not os.path.exists(dirpath_out_):
    os.mkdir(dirpath_out_)

  # Copy all avatars assigned to this class to the created folder
  for ind, res in res_pd_.iterrows():
    fname = res.loc['Filename']
    fpath_in = os.path.join(dirpath_in, fname)
    fpath_out = os.path.join(dirpath_out_, fname)
    shutil.copyfile(fpath_in, fpath_out)

In [ ]:
print(data_name_z)

random_around_24Feb_z


In [ ]:
arch_name_z = data_name_z + '.zip'
arch_name_tri = data_name_tri + '.zip'

!zip -q -r /content/$arch_name_z /content/$data_name_z
!zip -q -r /content/$arch_name_tri /content/$data_name_tri

In [ ]:
from google.colab import files

files.download(os.path.join('/content', data_name_z + '.zip'))
files.download(os.path.join('/content', data_name_tri + '.zip'))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>